In [1]:
#!pip --quiet install transformers datasets torch

In [1]:
import transformers
import datasets
from transformers.pipelines.pt_utils import KeyDataset
from pprint import PrettyPrinter
from tqdm.auto import tqdm
import gzip
import json

pprint = PrettyPrinter(compact=True).pprint

In [ ]:
# data=[]
# with gzip.open("sorted_qa_binary_parsebank.tsv.gz", 'rt') as f:
#     for line in f:
#         line=line.rstrip("\n")
#         cols=line.split("\t")
#         data.append(cols)

# texts = [text[2] for text in data]
# ids = [text[0] for text in data]

In [2]:
# this works for parsebank
def load_dataset(name, files):
    dset = datasets.load_dataset(
        "csv",
        data_files={name: files},
        delimiter="\t",
        split=name, # so that it returns a dataset instead of datasetdict
        column_names=['id', 'label', 'text', 'prob1', 'prob2'],
        features=datasets.Features({    # Here we tell how to interpret the attributes
        "text":datasets.Value("string"),
        "label":datasets.ClassLabel(names=["QA_NEW", "NOT_QA"]),
        "id":datasets.Value("string"),
        "prob1":datasets.Value("float"),
        "prob2":datasets.Value("float"),
         }))
    return dset

In [4]:
# for the not_found files that came from the splitting
def load_dataset(name, files):
    dset = datasets.load_dataset(
        "csv",
        data_files={name: files},
        delimiter="\t",
        split=name, # so that it returns a dataset instead of datasetdict
        column_names=['id', 'text'],
        features=datasets.Features({    # Here we tell how to interpret the attributes
        "text":datasets.Value("string"),
        "id":datasets.Value("string")
         }))
    return dset

In [3]:
dataset = load_dataset("train", "qa-sorted/sorted_qa_binary_parsebank.tsv.gz") #qa-sorted/sorted_qa_binary_falcon1M--.tsv

Found cached dataset csv (/users/annieske/.cache/huggingface/datasets/csv/default-b202921bd9758045/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [4]:
def replace_n(example):
  example["text"] = example["text"].replace("\n", " ")
  example["text"] = example["text"].replace("\\n", " ")

  return example

dataset = dataset.map(replace_n)

Loading cached processed dataset at /users/annieske/.cache/huggingface/datasets/csv/default-b202921bd9758045/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-64a1815df4a04ae0.arrow


In [5]:
#question_classifier = transformers.pipeline("ner", model="../models/token-qa/question_model3", tokenizer="xlm-roberta-base", ignore_labels=[""],device=0) # make ignore labels empty to not ignore O
# simple aggregation has been best so far?
 #stride=50, 

#answer_classifier = transformers.pipeline("ner", model="../models/token-qa/answer_model", tokenizer="xlm-roberta-base", ignore_labels=[""],device=0)

token_classifier = transformers.pipeline("ner", model="../models/token-qa/joined_model_simplified", tokenizer="xlm-roberta-base",aggregation_strategy="simple", ignore_labels=[""],device=0)
#aggregation_strategy="simple"

In [6]:
#questions_data = question_classifier(KeyDataset(dataset, "text")) #texts instead of keydataset
#answers_data = answer_classifier(KeyDataset(dataset, "text"))
tokens = token_classifier(KeyDataset(dataset, "text"))

##  this is testing

In [ ]:
for idx, extracted_entities in enumerate(tokens):
    print("Original text:\n{}".format(dataset[idx]["text"]))
    print("Original id:\n{}".format(dataset[idx]["id"]))
    print("Extracted entities:")
    for entity in extracted_entities:
        print(entity)

In [ ]:
for idx, extracted_entities in enumerate(questions_data):
    print("Original text:\n{}".format(dataset[idx]["text"]))
    print("Original id:\n{}".format(dataset[idx]["id"]))
    print("Extracted entities:")
    for entity in extracted_entities:
        print(entity)

In [ ]:
for idx, extracted_entities in enumerate(answers_data):
    print("Original text:\n{}".format(dataset[idx]["text"]))
    print("Original id:\n{}".format(dataset[idx]["id"]))
    print("Extracted entities:")
    for entity in extracted_entities:
        print(entity)

## processing

In [7]:
#from tqdm import tqdm

texts = dataset["text"]
ids = dataset["id"]

In [10]:
questions = []
answers = []
not_found = []
for idx, (extracted_entities, text, idd) in tqdm(enumerate(zip(tokens, texts, ids)), total=len(tokens)):
    previous = ""
    question = []
    answer = []
  # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    # note, I don't think I need the previous anymore, this is good enough
    for entity in extracted_entities:
        temp_q = ""
        if entity["entity_group"] == "QUESTION":
            temp_q = dataset[idx]["text"][entity["start"]:entity["end"]] #entity["word"]
            if len(temp_q.split()) < 3 and previous == "ANSWER":
                answer[len(answer)-1] = answer[len(answer)-1] + temp_q
                
            question.append(temp_q)
            previous = "QUESTION"
        elif entity["entity_group"] == "ANSWER":
            temp_a = dataset[idx]["text"][entity["start"]:entity["end"]]
            if len(temp_a.split()) < 3 and previous == "QUESTION":
                question[len(question)-1] = question[len(question)-1] + temp_a
            answer.append(temp_a)
            previous = "ANSWER"
        elif entity["entity_group"] == "O":
            previous = "O"
    if not answer or not question:
        not_found.append([text, idd])
    questions.append(question)
    answers.append(answer)

  0%|          | 0/31655 [00:00<?, ?it/s]

In [ ]:
questions = []
for idx, extracted_entities in enumerate(tqdm(questions_data)):
    question2 = []
    previous = ""
    question = ""
    # PERHAPS INSTEAD OF WORD I SHOULD USE STOP AND START TO GET FROM THE TEXT
    # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    for entity in extracted_entities:
        if entity["entity_group"] == "B":
            question = dataset[idx]["text"][entity["start"]:entity["end"]] #entity["word"]
            previous = "B"
        # I should take into account that sometimes QI might be before QB but maybe this is more trustworthy
        elif previous == "B" and entity["entity_group"] == "I": #  or previous == "I" and entity["entity_group"] == "B"
            question = question + " " + dataset[idx]["text"][entity["start"]:entity["end"]]
            previous = "I"
        elif entity["entity_group"] == "O":
            previous = "O"
        # ALSO TODO, if there are two question answer pairs? how to do this then? add some sort of fail safe here then
        elif previous == "O" and entity["entity_group"] == "B":
            question2.append(question)
            question = dataset[idx]["text"][entity["start"]:entity["end"]]
    if len(question2):
        question2.append(question)
        questions.append(question2)
    else:
        questions.append(question)

In [8]:
answers = []
for i in range(len(questions_data)):
    answers.append("")

In [ ]:
answers = []
for idx, extracted_entities in enumerate(tqdm(answers_data)):
    answer2 = []
    previous = ""
    answer = ""
    # PERHAPS INSTEAD OF WORD I SHOULD USE STOP AND START TO GET FROM THE TEXT
    # ^ DID NOT FIX THE SPACING FUCK, probably actually impossible to fix :(
    # now manually add space to at least fix some cases but now some words have a space in the middle
    for entity in extracted_entities:
        if entity["entity_group"] == "B":
            answer = dataset[idx]["text"][entity["start"]:entity["end"]]
            previous = "B"
        elif previous == "B" and entity["entity_group"] == "I": #  or previous == "I" and entity["entity_group"] == "B"
            answer = answer + " " + dataset[idx]["text"][entity["start"]:entity["end"]]
            previous = "I"
        elif entity["entity_group"] == "O":
            previous = "O"
        # ALSO TODO, if there are two question answer pairs? how to do this then? add some sort of fail safe here then
        elif previous == "O" and entity["entity_group"] == "B":
            answer2.append(answer)
            answer = dataset[idx]["text"][entity["start"]:entity["end"]]
    if len(answer2):
        answer2.append(answer)
        answers.append(answer2)
    else:
        answers.append(answer)

In [9]:
# import csv

# with open("results/falcon_ner.tsv", 'w') as outfile:
#     writer = csv.writer(outfile, delimiter="\t")
#     header = ["id", "question", "answer"]
#     writer.writerow(header)
#     for t, d, q, a in zip(texts, ids, questions, answers):
#         #print("TEXT:", t)
#         #print("QUESTION:", q)
#         #print("ANSWER:", a)
#         #print("-------------")
#         line = [d, q, a]
#         writer.writerow(line)
        

## saving to file

In [ ]:
# json dictionary, then just put as text: {q1: "", a1 = "", q2 = "" ....} so inside text will be many pairs -> may be annoying to load back if there can be any number of qa stuff inside "text"
# but it's the best I have right now :D
multiple=open("results/recent/ner_parsebank.jsonl","wt")
print(len(ids), len(questions), len(answers))
for idd, que,ans in zip(ids, questions, answers):
    #print(text_id)
    save = {}
    tmp = {}
    if len(que) != 0 and len(ans) != 0:
        for i, (q, a) in enumerate(zip(que,ans)):
            tmp[f"q{i}"] = q
            tmp[f"a{i}"] = a
        save["id"] = idd
        save["qa"] = tmp
        line=json.dumps(save,ensure_ascii=False)
        print(line,file=multiple)
    else:
        continue

In [ ]:
import csv

with open("results/recent/ner_parsebank_notfound.tsv", 'w') as outfile:
    writer = csv.writer(outfile, delimiter="\t")
    header = ["id", "text"]
    writer.writerow(header)
    for t, d in not_found:
        if t == "":
            continue
        line = [d, t]
        writer.writerow(line)


In [ ]:
# reading back the file

